In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import torch
from rtseg.barcodedetect.networks import model_dict as barcode_model_dict
from rtseg.utils.hardware import get_device_str
from rtseg.utils.param_io import load_params
from pathlib import Path
from rtseg.barcodedetect.utils import YoloLiveAugmentations, YoloLiveUnAugmentations, plot_results_batch
from skimage.io import imread
import matplotlib.pyplot as plt
import cv2
import albumentations as A
from rtseg.barcodedetect.transforms import YoloAugmentations, YoloTestAugmentations
from rtseg.barcodedetect.datasets import BarcodeDataset, BarcodeTestDataset
import matplotlib.patches as patches
%matplotlib qt5

In [3]:
params_path = Path("/home/pk/Documents/rtseg/rtseg/resources/reference_params/reference_linux.yaml")

In [4]:
params = load_params(params_path, ref_type="expt")

In [5]:
barcode_params = params.BarcodeAndChannels

In [6]:
device = 'cuda:0'

In [7]:
barcode_model = barcode_model_dict[barcode_params.architecture]
anchor_sizes = barcode_params.model_params.anchors.sizes
strides = barcode_params.model_params.anchors.strides
num_classes = barcode_params.model_params.num_classes

In [8]:
anchors_list = [[anchor_sizes[i], anchor_sizes[i+1], anchor_sizes[i+2]] for i in range(0, len(anchor_sizes), 3)]

anchors_t = tuple(torch.tensor(anch).float().to(device=device) for anch in anchors_list)
strides_t = tuple(torch.tensor(stride).to(device=device) for stride in strides)
barcode_model = barcode_model.parse(anchors=anchors_list, num_classes=num_classes).to(device=device)

### Load yolo model weights into the model

In [9]:
barcode_model_path = Path("/mnt/sda1/REALTIME/network_expts/barcode_expts/2025-01-17_13-26-07/model.pth")

In [10]:
barcode_model.load_state_dict(torch.load(barcode_model_path, map_location=device))
barcode_model.eval()

YOLOv3(
  (dark53): Darknet53(
    (conv_1): Sequential(
      (conv_1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm_1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_1): LeakyReLU(negative_slope=0.1)
    )
    (block_1): ConvResBlock(
      (conv): Sequential(
        (conv_2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (batch_norm_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (leaky_2): LeakyReLU(negative_slope=0.1)
      )
      (repeat_list): ModuleList(
        (0): Sequential(
          (conv_3): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (batch_norm_3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (leaky_3): LeakyReLU(negative_slope=0.1)
          (conv_4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (batch_norm_4): BatchNorm2

In [12]:
image_path = Path("/mnt/sda1/REALTIME/data/test_data/EXP-24-CD0103/40x newchip testimages/_1/Default/phase/img_channel000_position000_time000000000_z000.tif")

image = imread(image_path).astype('float32')

In [13]:
pre_barcode_transforms = YoloLiveAugmentations()

In [14]:
barcode_sample = pre_barcode_transforms({'phase': image})

In [15]:
barcode_sample['phase'].shape

torch.Size([1, 256, 800])

In [16]:
plt.figure()
plt.imshow(barcode_sample['phase'][0], cmap='gray')
plt.show()

In [17]:
from rtseg.barcodedetect.utils import outputs_to_bboxes, non_max_suppression, to_cpu

In [18]:
barcode_pred = barcode_model(barcode_sample['phase'].unsqueeze(0).to(device))

In [19]:
bboxes = outputs_to_bboxes(barcode_pred, anchors_t, strides_t)

In [20]:
bboxes_cleaned = non_max_suppression(bboxes, conf_thres=0.25,
                                     iou_thres=0.45)
bboxes_barcode = [bbox.numpy() for bbox in bboxes_cleaned][0]

In [21]:
bboxes_barcode

array([[4.7629596e+02, 7.8996033e+01, 4.8608344e+02, 1.8558261e+02,
        8.5779959e-01, 0.0000000e+00],
       [1.6205576e+02, 6.6377373e+01, 1.7286716e+02, 1.7421210e+02,
        8.0090129e-01, 0.0000000e+00],
       [6.3333069e+02, 9.0044670e+01, 6.4350159e+02, 1.9209134e+02,
        7.2835815e-01, 0.0000000e+00],
       [3.1964246e+02, 7.1160767e+01, 3.2968225e+02, 1.7968309e+02,
        5.3137660e-01, 0.0000000e+00],
       [7.9122137e+02, 8.8817650e+01, 7.9977850e+02, 1.9532199e+02,
        3.9682052e-01, 0.0000000e+00]], dtype=float32)

In [22]:
yolo_img_size = barcode_params.img_size

In [23]:
yolo_datapoint = {
    'yolo_size': yolo_img_size,
    'bboxes': bboxes_barcode
}

In [24]:
raw_shape = image.shape

In [25]:
post_barcode_transformations = YoloLiveUnAugmentations(
    parameters = {'resize': {
        'height': raw_shape[0],
        'width': raw_shape[1],
        }
    }
)

In [26]:
bboxes_final = post_barcode_transformations(yolo_datapoint)

In [27]:
bboxes_final = sorted(bboxes_final, key=lambda x: x[0])

In [28]:
bboxes_final

[array([611.7605   , 212.61502  , 652.5735   , 558.02313  ,   0.8009013,
          0.       ], dtype=float32),
 array([1.2066503e+03, 2.2793683e+02, 1.2445505e+03, 5.7554742e+02,
        5.3137660e-01, 0.0000000e+00], dtype=float32),
 array([1.7980172e+03, 2.5303416e+02, 1.8349648e+03, 5.9444427e+02,
        8.5779959e-01, 0.0000000e+00], dtype=float32),
 array([2.3908232e+03, 2.8842435e+02, 2.4292185e+03, 6.1529254e+02,
        7.2835815e-01, 0.0000000e+00], dtype=float32),
 array([2.9868606e+03, 2.8449405e+02, 3.0191638e+03, 6.2564075e+02,
        3.9682052e-01, 0.0000000e+00], dtype=float32)]

In [29]:
fig, ax = plt.subplots()
ax.imshow(image, cmap='gray')
for row in bboxes_final:
    rect = patches.Rectangle((row[0], row[1]), row[2] - row[0], row[3] - row[1], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
plt.show()

In [38]:
handle

In [11]:
def process_image(model, image):

    return {
        'segmentation': None,
        'barcodes': None,
        'error': False
    }

In [7]:
def prepare_barcode_model():
    pass

In [8]:
def get_barcodes(model, image):
    

SyntaxError: incomplete input (3991420517.py, line 2)